In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("HW3-Pyspark").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

23/12/11 16:40:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import tarfile
from urllib.request import urlopen
import os

if not os.path.exists('reuters-data'):
    os.mkdir('reuters-data')
    
    resp = urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz')
    file = tarfile.open(fileobj=resp, mode="r|gz")
    file.extractall('reuters-data/')
    file.close()

os.listdir('reuters-data/')

['reut2-020.sgm',
 'README.txt',
 'reut2-012.sgm',
 'reut2-014.sgm',
 '.ipynb_checkpoints',
 'reut2-008.sgm',
 'reut2-005.sgm',
 'reut2-017.sgm',
 'reut2-000.sgm',
 'reut2-007.sgm',
 'reut2-016.sgm',
 'reut2-018.sgm',
 'cat-descriptions_120396.txt',
 'reut2-013.sgm',
 'reut2-002.sgm',
 'all-topics-strings.lc.txt',
 'reut2-003.sgm',
 'reut2-015.sgm',
 'reut2-019.sgm',
 'reut2-021.sgm',
 'reut2-010.sgm',
 'reut2-011.sgm',
 'all-places-strings.lc.txt',
 'all-orgs-strings.lc.txt',
 'feldman-cia-worldfactbook-data.txt',
 'reut2-009.sgm',
 'all-people-strings.lc.txt',
 'all-exchanges-strings.lc.txt',
 'reut2-001.sgm',
 'lewis.dtd',
 'reut2-006.sgm',
 'reut2-004.sgm']

In [3]:
# !pip3 install beautifulsoup4
from bs4 import BeautifulSoup
# !pip3 install pandas
import pandas as pd

data = []
for index in range(22):
    filename = 'reuters-data/reut2-{0}.sgm'.format(str(index).zfill(3))
    print(filename)
    with open(filename, 'r', encoding = 'utf-8', errors='ignore') as infile:
        soup = BeautifulSoup(infile, 'html.parser')
        for page in soup.find_all('reuters'):
            newid = page['newid'].strip()
            lewissplit = page['lewissplit'].strip()
            cgisplit = page['cgisplit'].strip()
            topicTag = page['topics'].strip()
            date = page.find('date').get_text().strip()
            places = page.find('places').get_text(separator=' ').strip()
            
            if page.find('dateline') is not None:
                dateline = page.find('dateline').get_text().replace(' -', '').strip()
            else: dateline = ''
            
            if page.find('topics') is not None:
                topic = page.find('topics').get_text().strip()
            else: topic = ''
                
            if page.find('title') is not None:
                title = page.find('title').get_text().strip()
            else: title = ''
                
            if page.find('body') is not None:
                body = page.find('body').get_text().strip()
            else: body = ''
                
            data.append([newid, lewissplit, cgisplit, topicTag, date, places, dateline, topic, title, body])
        
pReuters = pd.DataFrame(data, columns=['Newid', 'lewissplit', 'cgisplit', 'topicTag', 'Date', 'Places', 'Dateline', 'Topic', 'Title', 'Article'])
print(pReuters.dtypes)
pReuters

reuters-data/reut2-000.sgm
reuters-data/reut2-001.sgm
reuters-data/reut2-002.sgm
reuters-data/reut2-003.sgm
reuters-data/reut2-004.sgm
reuters-data/reut2-005.sgm
reuters-data/reut2-006.sgm
reuters-data/reut2-007.sgm
reuters-data/reut2-008.sgm
reuters-data/reut2-009.sgm
reuters-data/reut2-010.sgm
reuters-data/reut2-011.sgm
reuters-data/reut2-012.sgm
reuters-data/reut2-013.sgm
reuters-data/reut2-014.sgm
reuters-data/reut2-015.sgm
reuters-data/reut2-016.sgm
reuters-data/reut2-017.sgm
reuters-data/reut2-018.sgm
reuters-data/reut2-019.sgm
reuters-data/reut2-020.sgm
reuters-data/reut2-021.sgm
Newid         object
lewissplit    object
cgisplit      object
topicTag      object
Date          object
Places        object
Dateline      object
Topic         object
Title         object
Article       object
dtype: object


,Newid,lewissplit,cgisplit,topicTag,Date,Places,Dateline,Topic,Title,Article
0,1,TRAIN,TRAINING-SET,YES,26-FEB-1987 15:01:01.79,el-salvador usa uruguay,"SALVADOR, Feb 26",cocoa,BAHIA COCOA REVIEW,Showers continued throughout the week in\nthe ...
1,2,TRAIN,TRAINING-SET,NO,26-FEB-1987 15:02:20.00,usa,"CLEVELAND, Feb 26",,STANDARD OIL <SRD> TO FORM FINANCIAL UNIT,Standard Oil Co and BP North America\nInc said...
2,3,TRAIN,TRAINING-SET,NO,26-FEB-1987 15:03:27.51,usa,"HOUSTON, Feb 26",,TEXAS COMMERCE BANCSHARES <TCB> FILES PLAN,Texas Commerce Bancshares Inc's Texas\nCommerc...
3,4,TRAIN,TRAINING-SET,NO,26-FEB-1987 15:07:13.72,usa brazil,"LOS ANGELES, Feb 26",,TALKING POINT/BANKAMERICA <BAC> EQUITY OFFER,BankAmerica Corp is not under\npressure to act...
4,5,TRAIN,TRAINING-SET,YES,26-FEB-1987 15:10:44.60,usa,"WASHINGTON, Feb 26",grainwheatcornbarleyoatsorghum,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE,The U.S. Agriculture Department\nreported the ...
...,...,...,...,...,...,...,...,...,...,...
21573,21574,TEST,TRAINING-SET,YES,19-OCT-1987 00:34:08.94,hong-kong japan india pakistan iran iraq,"HONG KONG, Oct 19",ship,JAPAN/INDIA CONFERENCE CUTS GULF WAR RISK CHARGES,The Japan/India-Pakistan-Gulf/Japan\nshipping ...
21574,21575,TEST,TRAINING-SET,YES,19-OCT-1987 00:18:22.79,ussr,"MOSCOW, Oct 19",ipi,SOVIET INDUSTRIAL GROWTH/TRADE SLOWER IN 1987,The Soviet Union's industrial output is\ngrowi...
21575,21576,TEST,TRAINING-SET,YES,19-OCT-1987 00:05:11.26,south-africa,"JOHANNESBURG, Oct 19",gold,SIX KILLED IN SOUTH AFRICAN GOLD MINE ACCIDENT,Six black miners have been killed\nand two inj...
21576,21577,TEST,TRAINING-SET,NO,19-OCT-1987 00:03:21.69,switzerland,"BERNE, Oct 19",,PROJECTIONS SHOW SWISS VOTERS WANT TRIED PARTIES,The prospect of a dominant alliance of\nsocial...


In [4]:
import string, re
!pip3 install contractions
!pip3 install num2words
!pip3 install nltk
import contractions
import num2words
import nltk
from nltk.corpus import stopwords
#nltk.download("stopwords")

def textTreprocess(text):
    
    #replace number with words
    #text = re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), text)
    
    #replace contractions
    text = contractions.fix(text)
    
    #text = text.lower()
    #text = re.sub(r'[^A-Za-z0-9]+', ' ', text)
    #text = re.sub("\s+", " ", text)
    text = text.replace('\n',' ')
    text = text.strip()
    return text

#pReuters.Article.apply(textTreprocess)

pReuters.Article = pReuters.Article.apply(textTreprocess)
pReuters.Article

0        Showers continued throughout the week in the B...
1        Standard Oil Co and BP North America Inc said ...
2        Texas Commerce Bancshares Inc's Texas Commerce...
3        BankAmerica Corp is not under pressure to act ...
4        The YOU.S. Agriculture Department reported the...
                               ...                        
21573    The Japan/India-Pakistan-Gulf/Japan shipping c...
21574    The Soviet Union's industrial output is growin...
21575    Six black miners have been killed and two inju...
21576    The prospect of a dominant alliance of sociali...
21577    The American Stock Exchange said it has introd...
Name: Article, Length: 21578, dtype: object

In [5]:
#Spark dataframe
pd.DataFrame.iteritems = pd.DataFrame.items
reuters = spark.createDataFrame(pReuters)
    
reuters.show()

/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/types.py:238: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(s.dtype):
23/12/11 16:41:30 WARN TaskSetManager: Stage 0 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.


+-----+----------+------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Newid|lewissplit|    cgisplit|topicTag|                Date|              Places|            Dateline|               Topic|               Title|             Article|
+-----+----------+------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    1|     TRAIN|TRAINING-SET|     YES|26-FEB-1987 15:01...|el-salvador usa u...|    SALVADOR, Feb 26|               cocoa|  BAHIA COCOA REVIEW|Showers continued...|
|    2|     TRAIN|TRAINING-SET|      NO|26-FEB-1987 15:02...|                 usa|   CLEVELAND, Feb 26|                    |STANDARD OIL <SRD...|Standard Oil Co a...|
|    3|     TRAIN|TRAINING-SET|      NO|26-FEB-1987 15:03...|                 usa|     HOUSTON, Feb 26|                    |TEXAS COMMERCE BA...|Texas Commerce Ba...

In [6]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, NGram
from pyspark.sql.functions import col, udf, split, explode, first
from pyspark.sql.types import IntegerType

#Split strings into tokens
regexTokenizer = RegexTokenizer(inputCol="Article", outputCol="Words", pattern="\\W")

reuters = regexTokenizer.transform(reuters)
reuters.select("Words").show(truncate=False)

23/12/11 16:41:35 WARN TaskSetManager: Stage 1 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
#Remove stop words
remover = StopWordsRemover(inputCol="Words", outputCol="newWords")
reuters = remover.transform(reuters)
reuters.select("newWords").show(truncate=False)

23/12/11 16:41:40 WARN TaskSetManager: Stage 2 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### K-shingles and output the set representation of the text dataset as a matrix

In [8]:
#Generate shingle
ngram = NGram(n=10, inputCol="newWords", outputCol="Shingles")
reuters = ngram.transform(reuters)
reuters.select("Shingles").show(truncate=False)

23/12/11 16:41:41 WARN TaskSetManager: Stage 3 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
##Shingle check
#reuters.withColumn('Shingle',(explode(col('Shingles'))))\
#    .groupBy('Shingle')\
#    .count()\
#    .sort('count', ascending=False)\
#    .show(50)

In [10]:
#Shingle check
#reuters.withColumn('Shingle',(explode(col('Shingles'))))\
#    .groupBy('Newid','Shingle')\
#    .count()\
#    .sort('count', ascending=False)\
#    .show(50)

In [11]:
"""reuters.withColumn('Shingles',(explode(col('Shingles'))))\
    .select('Newid','Shingles')\
    .stat.crosstab('Shingles','Newid')\
    .show()"""

reuters.filter(reuters.Newid < 300)\
    .withColumn('Shingles',(explode(col('Shingles'))))\
    .stat.crosstab('Shingles','Newid')\
    .show()

23/12/11 16:41:43 WARN TaskSetManager: Stage 4 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.
23/12/11 16:41:52 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+--

#### Minhash signatures

In [12]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

#reuters.filter(reuters.Newid < 300)\
#    .withColumn('Shingles',(explode(col('Shingles'))))\
#    .withColumn('id', F.dense_rank().over(Window.orderBy('Shingles')))\
#    .stat.crosstab('id','Newid')\
#    .show()

In [13]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="Shingles", outputCol="Vectors")
model = cv.fit(reuters)
reuters = model.transform(reuters)

print(model.getVocabSize())
#reuters.select("Vectors").show()

23/12/11 16:42:28 WARN TaskSetManager: Stage 14 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.


262144


In [14]:
#print((reuters.select("Vectors").count(), len(reuters.select("Vectors").columns)))

In [15]:
"""from pyspark.sql.types import *
vector_udf = udf(lambda vector: vector.toArray().tolist(),ArrayType(DoubleType()))
colvalues = reuters.select(vector_udf('Vectors').alias('Vectors'))
colvalues.collect()"""

"""vectors = reuters.select("Vectors").rdd.map(lambda row: row.Vectors.toArray())
from pyspark.mllib.linalg.distributed import RowMatrix, IndexedRowMatrix
matrix = RowMatrix(vectors)
matrix1 = IndexedRowMatrix(vectors)"""

'vectors = reuters.select("Vectors").rdd.map(lambda row: row.Vectors.toArray())\nfrom pyspark.mllib.linalg.distributed import RowMatrix, IndexedRowMatrix\nmatrix = RowMatrix(vectors)\nmatrix1 = IndexedRowMatrix(vectors)'

In [16]:
#print(matrix.numRows())
#matrix.rows.collect()
#matrix1.toBlockMatrix().show()

In [17]:
#Buketed Hashes
from pyspark.ml.feature import BucketedRandomProjectionLSH

brp = BucketedRandomProjectionLSH(inputCol="Vectors", outputCol="Hashes", bucketLength=5, numHashTables=5)

model = brp.fit(reuters)
reuters = model.transform(reuters)
reuters.select("Hashes").show(truncate=False)

23/12/11 16:42:50 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:42:50 WARN TaskSetManager: Stage 18 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.


+---------------------------------------+
|Hashes                                 |
+---------------------------------------+
|[[-1.0], [-1.0], [-1.0], [-1.0], [0.0]]|
|[[0.0], [-1.0], [0.0], [-1.0], [0.0]]  |
|[[0.0], [0.0], [0.0], [0.0], [0.0]]    |
|[[-1.0], [-1.0], [0.0], [-1.0], [-1.0]]|
|[[0.0], [0.0], [-1.0], [0.0], [-1.0]]  |
|[[0.0], [-1.0], [-1.0], [-1.0], [-1.0]]|
|[[0.0], [-1.0], [-1.0], [-1.0], [0.0]] |
|[[-1.0], [0.0], [0.0], [-1.0], [-1.0]] |
|[[0.0], [0.0], [-1.0], [0.0], [0.0]]   |
|[[-1.0], [0.0], [0.0], [-1.0], [0.0]]  |
|[[0.0], [0.0], [0.0], [-1.0], [0.0]]   |
|[[0.0], [-1.0], [0.0], [0.0], [-1.0]]  |
|[[0.0], [0.0], [-1.0], [0.0], [-1.0]]  |
|[[-1.0], [-1.0], [0.0], [0.0], [0.0]]  |
|[[-1.0], [0.0], [-1.0], [-1.0], [-1.0]]|
|[[-1.0], [-1.0], [0.0], [-1.0], [-1.0]]|
|[[0.0], [-1.0], [-1.0], [0.0], [0.0]]  |
|[[-1.0], [-1.0], [0.0], [-1.0], [0.0]] |
|[[0.0], [-1.0], [0.0], [0.0], [0.0]]   |
|[[0.0], [-1.0], [0.0], [0.0], [-1.0]]  |
+---------------------------------

In [18]:
#print((reuters.count(), len(reuters.columns)))

In [19]:
reuters.filter(reuters.Newid < 100)\
    .stat.crosstab('Hashes','Newid')\
    .show()

23/12/11 16:42:54 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:42:54 WARN TaskSetManager: Stage 19 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.
23/12/11 16:42:58 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:42:58 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:43:00 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:43:03 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:43:10 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB


+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|        Hashes_Newid|  1| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19|  2| 20| 21| 22| 23| 24| 25| 26| 27| 28| 29|  3| 30| 31| 32| 33| 34| 35| 36| 37| 38| 39|  4| 40| 41| 42| 43| 44| 45| 46| 47| 48| 49|  5| 50| 51| 52| 53| 54| 55| 56| 57| 58| 59|  6| 60| 61| 62| 63| 64| 65| 66| 67| 68| 69|  7| 70| 71| 72| 73| 74| 75| 76| 77| 78| 79|  8| 80| 81| 82| 83| 84| 85| 86| 87| 88| 89|  9| 90| 91| 92| 93| 94| 95| 96| 97| 98| 99|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+

#### LSH algorithm implementation with the resulting candidate pairs of similar documents. 

In [20]:
# Train and test
train = reuters.filter("lewissplit == 'TRAIN' and topicTag = 'YES'").select("Newid","Topic", "Vectors", "Hashes")
test = reuters.filter("lewissplit == 'TEST' and topicTag = 'YES' ").select("Newid","Topic", "Vectors", "Hashes")

In [21]:
print((train.count(), len(train.columns)))
print((test.count(), len(test.columns)))

23/12/11 16:43:18 WARN TaskSetManager: Stage 29 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.


(9603, 4)


23/12/11 16:43:19 WARN TaskSetManager: Stage 31 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.


(3299, 4)


In [22]:
#Approximately joining on Euclidean distance
prediction = model.approxSimilarityJoin(train, test, 1.5, distCol="EuclideanDistance")\
    .select(col("datasetA.Newid").alias("idTrain"),
            col("datasetB.Newid").alias("idTest"),
            col("datasetA.Topic").alias("topicTrain"),
            col("datasetB.Topic").alias("topicTest"),
            col("datasetA.Hashes").alias("hasheTrain"),
            col("datasetB.Hashes").alias("hasheTest"),
            col("EuclideanDistance"))
prediction.show()
#print((prediction.count(), len(prediction.columns)))

23/12/11 16:43:22 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:43:22 WARN TaskSetManager: Stage 33 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.
23/12/11 16:43:22 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:43:23 WARN TaskSetManager: Stage 34 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.
23/12/11 16:43:29 WARN DAGScheduler: Broadcasting large task binary with size 27.4 MiB
23/12/11 16:44:14 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB


+-------+------+----------+----------+--------------------+--------------------+------------------+
|idTrain|idTest|topicTrain| topicTest|          hasheTrain|           hasheTest| EuclideanDistance|
+-------+------+----------+----------+--------------------+--------------------+------------------+
|   5701| 19711|      earn|       acq|[[-1.0], [0.0], [...|[[-1.0], [0.0], [...|1.4142135623730951|
|   5722| 15153|      earn|      earn|[[-1.0], [-1.0], ...|[[-1.0], [-1.0], ...|1.4142135623730951|
|   5731| 16708|      earn|      earn|[[-1.0], [-1.0], ...|[[-1.0], [0.0], [...|1.4142135623730951|
|   5731| 15623|      earn|iron-steel|[[-1.0], [-1.0], ...|[[-1.0], [-1.0], ...|1.4142135623730951|
|   5731| 15780|      earn|      earn|[[-1.0], [-1.0], ...|[[-1.0], [0.0], [...|1.4142135623730951|
|   6173| 17671|      earn|       acq|[[-1.0], [-1.0], ...|[[-1.0], [-1.0], ...|1.4142135623730951|
|   6252| 18870|      earn|      earn|[[-1.0], [-1.0], ...|[[-1.0], [-1.0], ...|1.4142135623730951|


#### k-nearest neighbor 

In [23]:
#Approximate search of nearest neighbors
key = test.head()
print('Key: ', key)
key = key.Vectors
searchKNN = model.approxNearestNeighbors(test, key, 2).show()

23/12/11 16:44:15 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:44:15 WARN TaskSetManager: Stage 37 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.
23/12/11 16:44:15 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:44:16 WARN TaskSetManager: Stage 38 contains a task of very large size (5780 KiB). The maximum recommended task size is 1000 KiB.


Key:  Row(Newid='14826', Topic='trade', Vectors=SparseVector(262144, {80191: 1.0, 83110: 1.0, 84047: 1.0, 86088: 1.0, 86092: 1.0, 87421: 1.0, 88563: 1.0, 90002: 1.0, 93208: 1.0, 96840: 1.0, 101959: 1.0, 106036: 1.0, 107783: 1.0, 111559: 1.0, 118483: 1.0, 125815: 1.0, 128141: 1.0, 131533: 1.0, 132836: 1.0, 140719: 1.0, 140814: 1.0, 142627: 1.0, 145892: 1.0, 148360: 1.0, 149471: 1.0, 149913: 1.0, 151884: 1.0, 152253: 1.0, 152532: 1.0, 157733: 1.0, 158804: 1.0, 162335: 1.0, 165867: 1.0, 166853: 1.0, 169188: 1.0, 175971: 1.0, 176073: 1.0, 183347: 1.0, 183946: 1.0, 187673: 1.0, 190653: 1.0, 197446: 1.0, 199646: 1.0, 204958: 1.0, 206424: 1.0, 211507: 1.0, 213687: 1.0, 225075: 1.0, 226339: 1.0, 234965: 1.0, 242674: 1.0, 242897: 1.0, 245029: 1.0, 245385: 1.0, 249964: 1.0, 250072: 1.0, 258670: 1.0, 259019: 1.0, 260058: 1.0, 261104: 1.0}), Hashes=[DenseVector([-1.0]), DenseVector([0.0]), DenseVector([-1.0]), DenseVector([-1.0]), DenseVector([-1.0])])


23/12/11 16:44:18 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:44:18 WARN TaskSetManager: Stage 39 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.
23/12/11 16:44:22 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB


+-----+-----+--------------------+--------------------+-----------------+
|Newid|Topic|             Vectors|              Hashes|          distCol|
+-----+-----+--------------------+--------------------+-----------------+
|14826|trade|(262144,[80191,83...|[[-1.0], [0.0], [...|              0.0|
|14835|     |      (262144,[],[])|[[0.0], [0.0], [0...|7.745966692414834|
+-----+-----+--------------------+--------------------+-----------------+



#### Performance evaluation

In [24]:
#prediction.stat.crosstab('topicTrain','topicTest').show()

In [25]:
pPred = prediction.select('topicTrain', 'topicTest','hasheTrain','hasheTest').toPandas()
pPred

23/12/11 16:44:25 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:44:25 WARN TaskSetManager: Stage 41 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.
23/12/11 16:44:25 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
23/12/11 16:44:26 WARN TaskSetManager: Stage 42 contains a task of very large size (4498 KiB). The maximum recommended task size is 1000 KiB.
23/12/11 16:44:30 WARN DAGScheduler: Broadcasting large task binary with size 27.4 MiB
23/12/11 16:45:13 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB
                                                                                
Exception: java.lang.OutOfMemoryError thrown from the UncaughtExceptionHandler in thread "refresh progress"
Exception in thread "Spark Context Cleaner" java.lang.BootstrapMethodError: call site initialization exception
	at java.lang.invoke.CallSite.makeSite(CallSite.java:341)
	at java.l

Py4JJavaError: An error occurred while calling o293.collectToPython.
: java.lang.OutOfMemoryError: GC overhead limit exceeded


In [ ]:
index = 0
correct = 0
TP, FP, FN, TN = 0, 0, 0, 0
while index < len(pPred):
    if pPred['hasheTest'][index] == pPred['hasheTrain'][index] and pPred['topicTrain'][index] == pPred['topicTest'][index]:
        TP += 1
    elif pPred['hasheTest'][index] != pPred['hasheTrain'][index] and pPred['topicTrain'][index] == pPred['topicTest'][index]:
        FP += 1
    elif pPred['hasheTest'][index] == pPred['hasheTrain'][index] and pPred['topicTrain'][index] != pPred['topicTest'][index]:
        FN += 1
    elif pPred['hasheTest'][index] != pPred['hasheTrain'][index] and pPred['topicTrain'][index] != pPred['topicTest'][index]:
        TN += 1
        
    if pPred['topicTrain'][index] == pPred['topicTest'][index]:
        correct += 1
    
    index += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1score = 2 * (precision * recall) / (precision + recall)

print('Correct: ', correct, 'TP: ', TP, ',FP: ', FP, ',FN: ', FN, ',TN: ', TN)
print('accuracy: ', accuracy, ',precision: ', precision, ',recall: ', recall, ',F1score: ', F1score)